In [5]:
import pdb
import requests

## Helper Functions

In [72]:
def query_wiki_api(params):
    url="https://en.wikipedia.org/w/api.php"
    return requests.get(url, params).json()

def get_pages(resp):
    return list(resp['query']["pages"].keys())

def page_dicts(resp):
    pages = get_pages(resp)
    return [{"pageid": page} for page in pages]

def add_links(resp, pages):
    for article in pages:
        try:
            pageid = article['pageid']
            links = [link['title'] for link in resp['query']['pages'][pageid]['links']]
            if "links" in article:
                article['links'] += links
            else:
                article['links'] = links
        except:
            continue
            
    return pages

def add_linkshere(resp, pages):
    for article in pages:
        try:
            pageid = article['pageid']
            links = [link['title'] for link in resp['query']['pages'][pageid]['linkshere']]
            if "linkshere" in article:
                article['linkshere'] += links
            else:
                article['linkshere'] = links
        except:
            continue
            
    return pages

def add_categories(resp, pages):
    for article in pages:
        try:
            pageid = article['pageid']
            categories = [cat['title'] for cat in resp['query']['pages'][pageid]['categories']]
            if "categories" in article:
                article['categories'] += categories
            else:
                article['categories'] = categories
        except:
            continue
            
    return pages

def update_continue(resp, params):
    if resp.get("continue"):
        # remove any previous continue strings
        keys = list(params.keys())
        for key in keys:
            if "continue" in key:
                del params[key]
                
        # update with new params
        params.update(resp.get("continue"))        
        return params
    else:
        return False
    
        

In [73]:
class WikiMultiQuery:
    
    def __init__(self):
        pass
    
    def get(self, articles, params=None, pages=None):
        
        if not params:
            params = {
                "action": "query",
                "format": "json",

                "prop": "redirects|links|linkshere|categories",

                # redirects
                "rdnamespace": 0,
                "rdlimit": "max",

                # links
                "pllimit": "max",
                "plnamespace": 0,

                # linkshere
                "lhlimit": "max",
                "lhnamespace": 0,
                "lhshow": "!redirect",

                # categories
                "cllimit": "max",

                # automatic redirect
                "redirects": 1
            }
            
        
        
        params['titles'] = "|".join(articles)
        
        resp = query_wiki_api(params)
        
        if not pages:
            pages = page_dicts(resp)
        
        pages = add_links(resp, pages)
        pages = add_linkshere(resp, pages)
        pages = add_categories(resp, pages)
        
        # will return an updated params with continue statements OR False
        params = update_continue(resp, params)
        
        # if params still is truthy, then it was updated with a continue
        # start the process again on the continued work
        if params:
            return self.get(articles, params, pages)
        
        return pages

In [77]:
wmq = WikiMultiQuery()

wmq.get(["Random forest", "Decision tree", "Machine learning", "Sonata"])[1]['linkshere'][3]

'Aesthetics'